In [19]:
# -*- coding: utf-8 -*-
"""MidSem.ipynb
 
Automatically generated by Colaboratory.
 
Original file is located at
    https://colab.research.google.com/drive/1dB-4gzbC3DApvfjJHpRXXbdqoCAYdyie
"""
 
from nltk.tag import pos_tag
import nltk
from nltk.tag import BigramTagger
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix, recall_score, precision_score
from nltk.corpus import indian
from nltk.tag import tnt
import random
import pandas as pd
 
nltk.download('averaged_perceptron_tagger')
nltk.download('indian')
 
tag = []
bi_word = []
dataset = []
with open("/content/drive/MyDrive/sharesd/sms-cw_2/CN-MWE-Dataset-from-wordnet.txt") as file:
  dataset = file.readlines()
#print(dataset)
 
for row in dataset:
  row = row.split("\t")
  count = 0
  for i in row[1]:
    if i == "_":
      count += 1
  if count > 1:
    continue
  count = 0
  for i in row[1]:
    if i == "-":
      count += 1
  if count > 1:
    continue
  if "_" in row[1]:
    old = len(bi_word)
    bi_word.append(row[1].split("_"))
    new = len(bi_word)
    if new > old:
      tag.append(row[0].split("+"))
  elif "-" in row[1]:
    old = len(bi_word)
    bi_word.append(row[1].split("-"))
    new = len(bi_word)
    if new > old:
      tag.append(row[0].split("+"))
#print(tag)
#print(bi_word)
 
#print(len(tag), len(bi_word))
 
with open("/content/drive/MyDrive/sharesd/sms-cw_2/LVC-MWE-Dataset-from-wordnet.txt") as file:
  dataset = file.readlines()
#print(dataset)
 
for row in dataset:
  row = row.split("\t")
  count = 0
  for i in row[1]:
    if i == "_":
      count += 1
  if count > 1:
    continue
  count = 0
  for i in row[1]:
    if i == "-":
      count += 1
  if count > 1:
    continue
 
  if "_" in row[1]:
    old = len(bi_word)
    bi_word.append(row[1].split("_"))
    new = len(bi_word)
    if new > old:
      tag.append(row[0].split("+"))
  elif "-" in row[1]:
    old = len(bi_word)
    bi_word.append(row[1].split("-"))
    new = len(bi_word)
    if new > old:
      tag.append(row[0].split("+"))
#print(tag)
#print(bi_word)
wordnet_wrd = bi_word.copy
# Trainining the Dataset
dataset = []
#print(len(tag), len(bi_word))
for i in range(len(tag)):
  current_set = []
  first_pair = (bi_word[i][0], tag[i][0])
  second_pair = (bi_word[i][1], tag[i][1])
  dataset.append([first_pair, second_pair])
#print(dataset[0])
 
random.shuffle(dataset)
#print(dataset)
 
train_data = dataset[0: 20000]
test_data = dataset[20000: ]
 
pos_tagger = tnt.TnT()
pos_tagger.train(train_data)
 
print("POS TAGGER ACCURACY = {}".format(pos_tagger.evaluate(test_data)))
 
actual_tags = []
for val in dataset[20000:]:
  #print((val[0][0], val[1][0]))
  actual_tags.append((val[0][1], val[1][1]))
 
#print(actual_tags)
print("ALL DIFFERENT TAGS PAIR = {}".format(set(actual_tags)))
 
pos_tags = []
for val in dataset[20000: ]:
  curr_word = val[0][0] + " " +  val[1][0]
  pos_tags.append((pos_tagger.tag(curr_word.split(" "))[0][1], pos_tagger.tag(curr_word.split(" "))[1][1]))
print(pos_tags)
 
bigram_tagger = BigramTagger(train=dataset[0: 20000])
print("Accuracy of Bigram Tagging = {}".format(bigram_tagger.evaluate(dataset[20000:])))
 
bigram_tags = []
for val in dataset[20000: ]:
  curr_word = val[0][0] + " " +  val[1][0]
  bigram_tags.append((bigram_tagger.tag(curr_word.split(" "))[0][1], bigram_tagger.tag(curr_word.split(" "))[1][1]))
print(bigram_tags)
 
precision_1 = []
precision_2 = []
recall_1 = []
recall_2 = []
accuray_1 = []
accuray_2 = []
count_1 = []
count_2 = []
actual = []
 
ALL_DIFFERENT_TAGS_PAIR = {('n', 'v'), ('adj', 'v'), ('adv', 'n'), ('adj', 'n'), ('adv', 'v'), ('n', 'n')}
 
# Noun(n) + Verb(v)
count_actual = []
count_pos_tagger_nltk = []
count_bigram = []
 
for i in range(len(actual_tags)):
  if "n" in actual_tags[i][0] and "v" in actual_tags[i][1]:
    count_actual.append(1)
  else:
    count_actual.append(0)
  if "n" in pos_tags[i][0] and "v" in pos_tags[i][1]:
    count_pos_tagger_nltk.append(1)
  else:
    count_pos_tagger_nltk.append(0)
  
  if bigram_tags[i][0] == None or bigram_tags[i][1] == None:
    count_bigram.append(0)
    continue
  if ("n" in bigram_tags[i][0] and "v" in bigram_tags[i][1]):
    count_bigram.append(1)
  else:
    count_bigram.append(0)
 
count_pos_tagger = 0
count_bigram_tagger = 0
for i in count_pos_tagger_nltk:
  count_pos_tagger += i
for i in count_bigram:
  count_bigram_tagger += i
 
 
accuracy_pos_tagger = accuracy_score(count_actual, count_pos_tagger_nltk)
accuracy_bigram_tagger = accuracy_score(count_actual, count_bigram)
precision_pos_tagger = precision_score(count_actual, count_pos_tagger_nltk)
precision_bigram_tagger = precision_score(count_actual, count_bigram)
recall_pos_tagger = recall_score(count_actual, count_pos_tagger_nltk)
recall_bigram_tagger = recall_score(count_actual, count_bigram)
 
precision_1.append(precision_pos_tagger)
precision_2.append(precision_bigram_tagger)
recall_1.append(recall_pos_tagger)
recall_2.append(recall_bigram_tagger)
accuray_1.append(accuracy_pos_tagger)
accuray_2.append(accuracy_bigram_tagger)
count_1.append(count_pos_tagger)
count_2.append(count_bigram_tagger)
actual.append(count_actual.count(1))
print("=================== POS TAGGER ====================")
print("Count of Actual (Noun and Verbs pair) =", count_actual.count(1))
print("Count of PosTagger (Noun and Verbs pair) =", count_pos_tagger)
print("accuracy =", accuracy_pos_tagger)
print("precision =", precision_pos_tagger)
print("recall =", recall_pos_tagger)
 
print()
print("=================== BIGRAM TAGGER ====================")
print("Count of Actual (Noun and Verbs pair) =", count_actual.count(1))
print("Count of Bigram Tagger (Noun and Verbs pair) =", count_bigram_tagger)
print("accuracy =", accuracy_bigram_tagger)
print("precision =", precision_bigram_tagger)
print("recall =", recall_bigram_tagger)
 
# Adjective(adj) + Verb(v)
count_actual = []
count_pos_tagger_nltk = []
count_bigram = []
 
for i in range(len(actual_tags)):
  if "adj" in actual_tags[i][0] and "v" in actual_tags[i][1]:
    count_actual.append(1)
  else:
    count_actual.append(0)
  if "adj" in pos_tags[i][0] and "v" in pos_tags[i][1]:
    count_pos_tagger_nltk.append(1)
  else:
    count_pos_tagger_nltk.append(0)
  
  if bigram_tags[i][0] == None or bigram_tags[i][1] == None:
    count_bigram.append(0)
    continue
  if ("adj" in bigram_tags[i][0] and "v" in bigram_tags[i][1]):
    count_bigram.append(1)
  else:
    count_bigram.append(0)
 
count_pos_tagger = 0
count_bigram_tagger = 0
for i in count_pos_tagger_nltk:
  count_pos_tagger += i
for i in count_bigram:
  count_bigram_tagger += i
 
 
accuracy_pos_tagger = accuracy_score(count_actual, count_pos_tagger_nltk)
accuracy_bigram_tagger = accuracy_score(count_actual, count_bigram)
precision_pos_tagger = precision_score(count_actual, count_pos_tagger_nltk)
precision_bigram_tagger = precision_score(count_actual, count_bigram)
recall_pos_tagger = recall_score(count_actual, count_pos_tagger_nltk)
recall_bigram_tagger = recall_score(count_actual, count_bigram)
 
precision_1.append(precision_pos_tagger)
precision_2.append(precision_bigram_tagger)
recall_1.append(recall_pos_tagger)
recall_2.append(recall_bigram_tagger)
accuray_1.append(accuracy_pos_tagger)
accuray_2.append(accuracy_bigram_tagger)
count_1.append(count_pos_tagger)
count_2.append(count_bigram_tagger)
actual.append(count_actual.count(1))
 
print("=================== POS TAGGER ====================")
print("Count of Actual (Adjective and Verbs pair) =", count_actual.count(1))
print("Count of PosTagger (Adjective and Verbs pair) =", count_pos_tagger)
print("accuracy =", accuracy_pos_tagger)
print("precision =", precision_pos_tagger)
print("recall =", recall_pos_tagger)
 
print()
print("=================== BIGRAM TAGGER ====================")
print("Count of Actual (Adjective and Verbs pair) =", count_actual.count(1))
print("Count of Bigram Tagger (Adjective and Verbs pair) =", count_bigram_tagger)
print("accuracy =", accuracy_bigram_tagger)
print("precision =", precision_bigram_tagger)
print("recall =", recall_bigram_tagger)
 
# Adjective(adj) + Noun(n)
count_actual = []
count_pos_tagger_nltk = []
count_bigram = []
 
for i in range(len(actual_tags)):
  if "adj" in actual_tags[i][0] and "n" in actual_tags[i][1]:
    count_actual.append(1)
  else:
    count_actual.append(0)
  if "adj" in pos_tags[i][0] and "n" in pos_tags[i][1]:
    count_pos_tagger_nltk.append(1)
  else:
    count_pos_tagger_nltk.append(0)
  
  if bigram_tags[i][0] == None or bigram_tags[i][1] == None:
    count_bigram.append(0)
    continue
  if ("adj" in bigram_tags[i][0] and "n" in bigram_tags[i][1]):
    count_bigram.append(1)
  else:
    count_bigram.append(0)
 
count_pos_tagger = 0
count_bigram_tagger = 0
for i in count_pos_tagger_nltk:
  count_pos_tagger += i
for i in count_bigram:
  count_bigram_tagger += i
 
 
accuracy_pos_tagger = accuracy_score(count_actual, count_pos_tagger_nltk)
accuracy_bigram_tagger = accuracy_score(count_actual, count_bigram)
precision_pos_tagger = precision_score(count_actual, count_pos_tagger_nltk)
precision_bigram_tagger = precision_score(count_actual, count_bigram)
recall_pos_tagger = recall_score(count_actual, count_pos_tagger_nltk)
recall_bigram_tagger = recall_score(count_actual, count_bigram)
 
precision_1.append(precision_pos_tagger)
precision_2.append(precision_bigram_tagger)
recall_1.append(recall_pos_tagger)
recall_2.append(recall_bigram_tagger)
accuray_1.append(accuracy_pos_tagger)
accuray_2.append(accuracy_bigram_tagger)
count_1.append(count_pos_tagger)
count_2.append(count_bigram_tagger)
actual.append(count_actual.count(1))
 
print("=================== POS TAGGER ====================")
print("Count of Actual (Adjective and Noun pair) =", count_actual.count(1))
print("Count of PosTagger (Adjective and Noun pair) =", count_pos_tagger)
print("accuracy =", accuracy_pos_tagger)
print("precision =", precision_pos_tagger)
print("recall =", recall_pos_tagger)
 
print()
print("=================== BIGRAM TAGGER ====================")
print("Count of Actual (Adjective and Noun pair) =", count_actual.count(1))
print("Count of Bigram Tagger (Adjective and Noun pair) =", count_bigram_tagger)
print("accuracy =", accuracy_bigram_tagger)
print("precision =", precision_bigram_tagger)
print("recall =", recall_bigram_tagger)
 
# Adjverb(adv) + Noun(n)
count_actual = []
count_pos_tagger_nltk = []
count_bigram = []
 
for i in range(len(actual_tags)):
  if "adv" in actual_tags[i][0] and "n" in actual_tags[i][1]:
    count_actual.append(1)
  else:
    count_actual.append(0)
  if "adv" in pos_tags[i][0] and "n" in pos_tags[i][1]:
    count_pos_tagger_nltk.append(1)
  else:
    count_pos_tagger_nltk.append(0)
  
  if bigram_tags[i][0] == None or bigram_tags[i][1] == None:
    count_bigram.append(0)
    continue
  if ("adv" in bigram_tags[i][0] and "n" in bigram_tags[i][1]):
    count_bigram.append(1)
  else:
    count_bigram.append(0)
 
count_pos_tagger = 0
count_bigram_tagger = 0
for i in count_pos_tagger_nltk:
  count_pos_tagger += i
for i in count_bigram:
  count_bigram_tagger += i
 
 
accuracy_pos_tagger = accuracy_score(count_actual, count_pos_tagger_nltk)
accuracy_bigram_tagger = accuracy_score(count_actual, count_bigram)
precision_pos_tagger = precision_score(count_actual, count_pos_tagger_nltk)
precision_bigram_tagger = precision_score(count_actual, count_bigram)
recall_pos_tagger = recall_score(count_actual, count_pos_tagger_nltk)
recall_bigram_tagger = recall_score(count_actual, count_bigram)
 
precision_1.append(precision_pos_tagger)
precision_2.append(precision_bigram_tagger)
recall_1.append(recall_pos_tagger)
recall_2.append(recall_bigram_tagger)
accuray_1.append(accuracy_pos_tagger)
accuray_2.append(accuracy_bigram_tagger)
count_1.append(count_pos_tagger)
count_2.append(count_bigram_tagger)
actual.append(count_actual.count(1))
 
print("=================== POS TAGGER ====================")
print("Count of Actual (Adverb and Noun pair) =", count_actual.count(1))
print("Count of PosTagger (Adverb and Noun pair) =", count_pos_tagger)
print("accuracy =", accuracy_pos_tagger)
print("precision =", precision_pos_tagger)
print("recall =", recall_pos_tagger)
 
print()
print("=================== BIGRAM TAGGER ====================")
print("Count of Actual (Adverb and Noun pair) =", count_actual.count(1))
print("Count of Bigram Tagger (Adverb and Noun pair) =", count_bigram_tagger)
print("accuracy =", accuracy_bigram_tagger)
print("precision =", precision_bigram_tagger)
print("recall =", recall_bigram_tagger)
 
# Adverb(adv) + Verb(v)
count_actual = []
count_pos_tagger_nltk = []
count_bigram = []
 
for i in range(len(actual_tags)):
  if "adv" in actual_tags[i][0] and "v" in actual_tags[i][1]:
    count_actual.append(1)
  else:
    count_actual.append(0)
  if "adv" in pos_tags[i][0] and "v" in pos_tags[i][1]:
    count_pos_tagger_nltk.append(1)
  else:
    count_pos_tagger_nltk.append(0)
  
  if bigram_tags[i][0] == None or bigram_tags[i][1] == None:
    count_bigram.append(0)
    continue
  if ("adv" in bigram_tags[i][0] and "v" in bigram_tags[i][1]):
    count_bigram.append(1)
  else:
    count_bigram.append(0)
 
count_pos_tagger = 0
count_bigram_tagger = 0
for i in count_pos_tagger_nltk:
  count_pos_tagger += i
for i in count_bigram:
  count_bigram_tagger += i
 
 
accuracy_pos_tagger = accuracy_score(count_actual, count_pos_tagger_nltk)
accuracy_bigram_tagger = accuracy_score(count_actual, count_bigram)
precision_pos_tagger = precision_score(count_actual, count_pos_tagger_nltk)
precision_bigram_tagger = precision_score(count_actual, count_bigram)
recall_pos_tagger = recall_score(count_actual, count_pos_tagger_nltk)
recall_bigram_tagger = recall_score(count_actual, count_bigram)
 
precision_1.append(precision_pos_tagger)
precision_2.append(precision_bigram_tagger)
recall_1.append(recall_pos_tagger)
recall_2.append(recall_bigram_tagger)
accuray_1.append(accuracy_pos_tagger)
accuray_2.append(accuracy_bigram_tagger)
count_1.append(count_pos_tagger)
count_2.append(count_bigram_tagger)
actual.append(count_actual.count(1))
 
print("=================== POS TAGGER ====================")
print("Count of Actual (Adverb and Verb pair) =", count_actual.count(1))
print("Count of PosTagger (Adverb and Verb pair) =", count_pos_tagger)
print("accuracy =", accuracy_pos_tagger)
print("precision =", precision_pos_tagger)
print("recall =", recall_pos_tagger)
 
print()
print("=================== BIGRAM TAGGER ====================")
print("Count of Actual (Adverb and Verb pair) =", count_actual.count(1))
print("Count of Bigram Tagger (Adverb and Verb pair) =", count_bigram_tagger)
print("accuracy =", accuracy_bigram_tagger)
print("precision =", precision_bigram_tagger)
print("recall =", recall_bigram_tagger)
 
# Noun(n) + Noun(n)
count_actual = []
count_pos_tagger_nltk = []
count_bigram = []
 
for i in range(len(actual_tags)):
  if "n" in actual_tags[i][0] and "n" in actual_tags[i][1]:
    count_actual.append(1)
  else:
    count_actual.append(0)
  if "n" in pos_tags[i][0] and "n" in pos_tags[i][1]:
    count_pos_tagger_nltk.append(1)
  else:
    count_pos_tagger_nltk.append(0)
  
  if bigram_tags[i][0] == None or bigram_tags[i][1] == None:
    count_bigram.append(0)
    continue
  if ("n" in bigram_tags[i][0] and "n" in bigram_tags[i][1]):
    count_bigram.append(1)
  else:
    count_bigram.append(0)
 
count_pos_tagger = 0
count_bigram_tagger = 0
for i in count_pos_tagger_nltk:
  count_pos_tagger += i
for i in count_bigram:
  count_bigram_tagger += i
 
 
accuracy_pos_tagger = accuracy_score(count_actual, count_pos_tagger_nltk)
accuracy_bigram_tagger = accuracy_score(count_actual, count_bigram)
precision_pos_tagger = precision_score(count_actual, count_pos_tagger_nltk)
precision_bigram_tagger = precision_score(count_actual, count_bigram)
recall_pos_tagger = recall_score(count_actual, count_pos_tagger_nltk)
recall_bigram_tagger = recall_score(count_actual, count_bigram)
 
precision_1.append(precision_pos_tagger)
precision_2.append(precision_bigram_tagger)
recall_1.append(recall_pos_tagger)
recall_2.append(recall_bigram_tagger)
accuray_1.append(accuracy_pos_tagger)
accuray_2.append(accuracy_bigram_tagger)
count_1.append(count_pos_tagger)
count_2.append(count_bigram_tagger)
actual.append(count_actual.count(1))
 
print("=================== POS TAGGER ====================")
print("Count of Actual (Noun and Noun pair) =", count_actual.count(1))
print("Count of PosTagger (Noun and Noun pair) =", count_pos_tagger)
print("accuracy =", accuracy_pos_tagger)
print("precision =", precision_pos_tagger)
print("recall =", recall_pos_tagger)
 
print()
print("=================== BIGRAM TAGGER ====================")
print("Count of Actual (Noun and Noun pair) =", count_actual.count(1))
print("Count of Bigram Tagger (Noun and Noun pair) =", count_bigram_tagger)
print("accuracy =", accuracy_bigram_tagger)
print("precision =", precision_bigram_tagger)
print("recall =", recall_bigram_tagger)
 
# POS TAGGER
data_frame = pd.DataFrame(actual, index=["Noun + Verb",
                                          "Adjective + verb", 
                                          "Adjective + Noun",
                                          "Adverb + Noun",
                                          "Adverb + Verb",
                                          "Noun + Noun"],
                          columns=["Actual"])
data_frame["count"] = count_1
data_frame["precision"] = precision_1
data_frame["recall"] = recall_1
data_frame["accuracy"] = accuray_1
#print("===================== BIGRAM TAGGED (using pos tagger) =====================")
print("===================== BIGRAM TAGGED (using pos tagger) =====================")
print(data_frame)
 
# POS TAGGER
data_frame = pd.DataFrame(actual, index=["Noun + Verb",
                                          "Adjective + verb", 
                                          "Adjective + Noun",
                                          "Adverb + Noun",
                                          "Adverb + Verb",
                                          "Noun + Noun"],
                          columns=["Actual"])
data_frame["count"] = count_2
data_frame["precision"] = precision_2
data_frame["recall"] = recall_2
data_frame["accuracy"] = accuray_2
print("===================== BIGRAM TAGGED (using bigram tagger) =====================")
print(data_frame)


bi_wordss = bi_word


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Package indian is already up-to-date!
POS TAGGER ACCURACY = 0.8158284023668639
ALL DIFFERENT TAGS PAIR = {('adj', 'v'), ('adv', 'v'), ('n', 'n'), ('adj', 'n'), ('adv', 'n'), ('n', 'v')}
[('n', 'n'), ('adv', 'n'), ('adj', 'n'), ('n', 'n'), ('Unk', 'v'), ('n', 'n'), ('n', 'n'), ('n', 'n'), ('Unk', 'Unk'), ('n', 'n'), ('Unk', 'n'), ('n', 'v'), ('n', 'n'), ('n', 'n'), ('n', 'Unk'), ('n', 'n'), ('n', 'n'), ('adj', 'n'), ('n', 'n'), ('adj', 'n'), ('Unk', 'n'), ('n', 'n'), ('n', 'n'), ('n', 'n'), ('n', 'n'), ('n', 'n'), ('Unk', 'n'), ('n', 'n'), ('n', 'Unk'), ('Unk', 'v'), ('n', 'n'), ('n', 'n'), ('n', 'v'), ('n', 'n'), ('n', 'n'), ('n', 'n'), ('n', 'n'), ('adj', 'Unk'), ('Unk', 'n'), ('n', 'v'), ('Unk', 'Unk'), ('Unk', 'v'), ('n', 'v

In [10]:
!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!pip install inltk

In [ ]:
from inltk.inltk import setup

setup('hi')

In [ ]:
# from inltk.inltk import get_embedding_vectors
# word_emds = []
# wor = bi_word.copy()
# for i in wor:
#   temp = i[0]+" "+i[1]
#   #word_emds.append(get_embedding_vectors(temp,"hi"))
# vec = get_embedding_vectors("मानव विज्ञानी","hi")

# print(len(vec[0]))
# print(len(word_emds))

In [20]:
import warnings
warnings.filterwarnings("ignore")

from inltk.inltk import get_sentence_similarity
wor = bi_word[:20]
for i in wor:
  bi = i[0]+ " " + i[1]
  sim_val = get_sentence_similarity('20वां वर्ष',bi,'hi')
  print(sim_val)
  if sim_val>0.9:
    print(bi)

#20वां वर्ष
#मानव विज्ञानी

0.8293665647506714


0.6775018572807312


0.8240184783935547


0.646725594997406


0.9473598003387451
20वाँ वर्ष


...

0.7199112772941589


0.853439211845398


0.727321982383728


0.8687464594841003


0.6901451349258423


In [12]:
import warnings
warnings.filterwarnings("ignore")
from inltk.inltk import get_sentence_similarity

def sim_words_in_dataset_inputtext(inp,bi_wordsss):
  word_s = bi_wordsss[:20]
  text_tokenss = []
  oup_bi = []
  text_tokenss.append(inp.split(" "))
  #print(text_tokenss[0])
  for k in range(len(text_tokenss[0])-1):
    for i in word_s:
      inp_bi = i[0]+ " " + i[1]
      dataset_bi = text_tokenss[0][k]+ " "+text_tokenss[0][k+1]
      sim_val = get_sentence_similarity(inp_bi,dataset_bi,'hi')
      #print(sim_val)
      if sim_val>0.98:
        #print(inp_bi)
        oup_bi.append(inp_bi)
  return oup_bi  
    
#print(sim_words_in_dataset_inputtext('हिंदी में टायपिंग करना 20वां वर्ष बहुत आसान बना ',bi_word)) #('मानव विज्ञानी')


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# wor = bi_word.copy()
# wor = np.array(wor)
# import operator
# def similarity_check(user, doc_topic_matrix):
#   docs_size = doc_topic_matrix.shape[0]
#   sim = {}
#   for i in range(0, docs_size):
#     sim[i] = float(cosine_similarity(user.reshape(1,-1), doc_topic_matrix[i].reshape(1,-1)))
#   sim = dict( sorted(sim.items(), key=operator.itemgetter(1),reverse=True))
#   return sim
# find_sim = ['मानव', 'विज्ञानी']
# find_sim = np.array(find_sim)
# #sim_scores = user_docs_sim(user_topic_matrix[1], doc_topic_matrix)
# sim_scores = similarity_check(find_sim,wor)
# sim_scores

In [13]:
#pos_tagger for LVC-MWE-Dataset-from-corpus.txt

tag = []
bi_word = []
dataset = []
with open("/content/drive/MyDrive/sharesd/sms-cw_2/LVC-MWE-Dataset-from-corpus.txt") as file:
  dataset = file.readlines()
#print(dataset)
 
for row in dataset:
  row = row.split("\t")
  count = 0
  for i in row[0]:
    if i == " ":
      count += 1
  if count > 1:
    continue
  count = 0
  for i in row[1]:
    if i == "-":
      count += 1
  if count > 1:
    continue
  if " " in row[0]:
    #old = len(bi_word)
    bi_word.append(row[0].split(" "))
    #new = len(bi_word)

print(bi_word)
final_tags = []
first_tag = []
second_tag = []
for ind in range(len(bi_word)):
  final_tags.append(pos_tagger.tag(bi_word[ind]))

print(final_tags)
for i in range(len(final_tags)):
  for j in range(len(final_tags[i])):
    if j%2==0:
      first_tag.append(final_tags[i][j][1])
    else:
      second_tag.append(final_tags[i][j][1])


k = len(first_tag)
bigram_note = []
for i in range(k):
  if first_tag[i] == 'n' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'n' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])

print("Total number of bi-words in the file: " + str(len(final_tags)))
print("Calculated Meaningful bigrams: " + str(len(bigram_note)))
print("The Meaningful Bigrams: ")
print(bigram_note)

[['अधिष्ठित', 'किया'], ['प्रविष्ट', 'हुआ'], ['जारी', 'किए'], ['मंडित', 'करेंगे'], ['पेश', 'किया'], ['खड़ा', 'करके'], ['तप्त', 'करके'], ['धुंधला', 'कर'], ['मंत्रमुग्ध', 'करने'], ['आकर्षित', 'करना'], ['प्रतिपादित', 'किये'], ['संरक्षित', 'किए'], ['गिरफ्तार', 'करते'], ['अदृश्य', 'होती'], ['क्षुब्ध', 'करने'], ['उद्वेलित', 'कर'], ['अभिहित', 'किए'], ['दर्ज', 'करनी'], ['दण्डित', 'करने'], ['उत्पन्न', 'करता'], ['एकत्र', 'करता'], ['गरम', 'करें'], ['प्रत्यक्ष', 'करता'], ['सावधान', 'करते'], ['बदनाम', 'करना'], ['अवरुद्ध', 'करके'], ['मनोनीत', 'करते'], ['घटित', 'हो'], ['प्रदर्शित', 'किए'], ['नष्ट', 'कर'], ['सुशोभित', 'करती'], ['फलित', 'करती'], ['सत्यापित', 'कर'], ['प्रकाशित', 'की'], ['छोटा', 'करके'], ['विकीर्ण', 'करते'], ['कुर्क', 'कर'], ['उत्पीड़ित', 'किये'], ['समान', 'कर'], ['अपदस्थ', 'कर'], ['चित्रित', 'किए'], ['शामिल', 'करती'], ['सच', 'कहा'], ['संबद्ध', 'करने'], ['अलग', 'करें'], ['उपेक्षित', 'किया'], ['खराब', 'कर'], ['संशोधित', 'करता'], ['संकीर्ण', 'कर'], ['आबद्ध', 'कर'], ['ताजा', 'करने'], ['रंजित

In [ ]:
#pos_tagger for CN-MWE-Dataset-from-corpus.txt

tag = []
bi_word = []
dataset = []
with open("/content/drive/MyDrive/sharesd/sms-cw_2/CN-MWE-Dataset-from-corpus.txt") as file:
  dataset = file.readlines()
#print(dataset)
 
for row in dataset:
  row = row.split("\t")
  count = 0
  for i in row[0]:
    if i == " ":
      count += 1
  if count > 1:
    continue
  count = 0
  for i in row[1]:
    if i == "-":
      count += 1
  if count > 1:
    continue
  if " " in row[0]:
    old = len(bi_word)
    bi_word.append(row[0].split(" "))
    new = len(bi_word)

final_tags = []
first_tag = []
second_tag = []
for ind in range(len(bi_word)):
  final_tags.append(pos_tagger.tag(bi_word[ind]))


for i in range(len(final_tags)):
  for j in range(len(final_tags[i])):
    if j%2==0:
      first_tag.append(final_tags[i][j][1])
    else:
      second_tag.append(final_tags[i][j][1])


k = len(first_tag)
bigram_note = []
for i in range(k):
  if first_tag[i] == 'n' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'n' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])

print("Total number of bi-words in the file: " + str(len(final_tags)))
print("Calculated Meaningful bigrams: " + str(len(bigram_note)))
print("The Meaningful Bigrams: ")
print(bigram_note)

Total number of bi-words in the file: 12000
Calculated Meaningful bigrams: 5192
The Meaningful Bigrams: 
['कंप्यूटर गेम', 'विक्टोरिया प्रांत', 'नेहरू परिवार', 'नमक कानून', 'कुर्ता पाजामा', 'दुर्गा मन्दिर', 'राजस्थान राज्य', 'व्यास नदी', 'तीर्थ स्थल', 'स्वर्ण पत्र', 'नागरिक सेना', 'डाटा सिक्युरिटी', 'शिशु विशेषज्ञ', 'जीव जंतु', 'संगीत आचार्य', 'प्रशांत महासागर', 'पश्चिम ऑस्ट्रेलिया', 'अध्ययन केन्द्र', 'रेलवे पुलिस', 'नीलगिरि श्रृंखला', 'रिक्शा चालक', 'ध्यान केन्द्र', 'नेहरू बाजार', 'वन संसार', 'हाथी दांत', 'अजमेर पर्यटन', 'राम मंदिर', 'वर्तमान युग', 'हॉकी खिलाड़ी', 'क्रिकेट मैच', 'नेहरू पार्क', 'आपात बैठक', 'कला केंद्र', 'सर्राफा बाजार', 'आज्ञा पत्र', 'जावा द्वीप', 'केन्द्र सरकार', 'कार्यकारी समिति', 'भगवान दत्तात्रेय', 'चन्द्र भगवान', 'टेस्ट मैच', 'खाड़ी युद्ध', 'कृषि परिषद', 'चीनी उद्योग', 'कलिंग साम्राज्य', 'जमीन विवाद', 'यात्रा उद्योग', 'ब्राह्मणी चील', 'पश्चिम जावा', 'भारत सरकार', 'ज़िला मुख्यालय', 'सब्जी बाजार', 'नृत्य संगीत', 'बुनाई केन्द्र', 'कृषि केन्द्र', 'शोध समिति', 'सहायता 

In [ ]:
#bigram_tagger for LVC-MWE-Dataset-from-corpus.txt

tag = []
bi_word = []
dataset = []
with open("/content/drive/MyDrive/sharesd/sms-cw_2/LVC-MWE-Dataset-from-corpus.txt") as file:
  dataset = file.readlines()
#print(dataset)
 
for row in dataset:
  row = row.split("\t")
  count = 0
  for i in row[0]:
    if i == " ":
      count += 1
  if count > 1:
    continue
  count = 0
  for i in row[1]:
    if i == "-":
      count += 1
  if count > 1:
    continue
  if " " in row[0]:
    old = len(bi_word)
    bi_word.append(row[0].split(" "))
    new = len(bi_word)


final_tags = []
first_tag = []
second_tag = []
for ind in range(len(bi_word)):
  final_tags.append(bigram_tagger.tag(bi_word[ind]))


for i in range(len(final_tags)):
  for j in range(len(final_tags[i])):
    if j%2==0:
      first_tag.append(final_tags[i][j][1])
    else:
      second_tag.append(final_tags[i][j][1])


k = len(first_tag)
bigram_note = []
for i in range(k):
  if first_tag[i] == 'n' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'n' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])

print("Total number of bi-words in the file: " + str(len(final_tags)))
print("Calculated Meaningful bigrams: " + str(len(bigram_note)))
print("The Meaningful Bigrams: ")
print(bigram_note)

#for i in bigram_note:
#  print(bi_word[i]+" "+bi_word[i+1])

Total number of bi-words in the file: 4000
Calculated Meaningful bigrams: 245
The Meaningful Bigrams: 
['आकर्षित करना', 'बदनाम करना', 'नष्ट कर', 'कुर्क कर', 'समान कर', 'खराब कर', 'संकीर्ण कर', 'प्रसन्न कर', 'पवित्र कर', 'कंपित कर', 'प्रज्वलित कर', 'हासिल करना', 'संचालित करना', 'निर्धारित कर', 'प्रविष्ट करना', 'क्रोधित कर', 'समाविष्ट करना', 'नाखुश कर', 'फीका कर', 'परिवर्तित कर', 'निमंत्रित कर', 'बेकार करना', 'जाग्रत करना', 'गोल करना', 'मूक कर', 'केंद्रित कर', 'बाध्य कर', 'चार कर', 'व्यतीत करना', 'ठंडा करना', 'गर्म कर', 'विकसित कर', 'रवाना कर', 'रद्द कर', 'बर्बाद कर', 'लिपिबद्ध कर', 'उन्मत्त कर', 'आलोकित कर', 'सक्रिय कर', 'प्रशिक्षित कर', 'समर्पित करना', 'तृप्त कर', 'पैदा करना', 'नाकाम करना', 'हाजिर कर', 'कायम करना', 'परिरक्षित करना', 'नामजद कर', 'शिथिल कर', 'निर्वाचित कर', 'उत्तीर्ण करना', 'चिकना कर', 'परास्त कर', 'निर्भर करना', 'कुल कर', 'जीवित कर', 'पागल कर', 'तबाह कर', 'बेनकाब कर', 'जिंदा करना', 'कोमल कर', 'शोषित कर', 'हराम कर', 'बरबाद करना', 'साफ़ कर', 'विवश कर', 'पूर्ण कर', 'आसान क

In [ ]:
#bigram_tagger for CN-MWE-Dataset-from-corpus.txt

tag = []
bi_word = []
dataset = []
with open("/content/drive/MyDrive/sharesd/sms-cw_2/CN-MWE-Dataset-from-corpus.txt") as file:
  dataset = file.readlines()
#print(dataset)
 
for row in dataset:
  row = row.split("\t")
  count = 0
  for i in row[0]:
    if i == " ":
      count += 1
  if count > 1:
    continue
  count = 0
  for i in row[1]:
    if i == "-":
      count += 1
  if count > 1:
    continue
  if " " in row[0]:
    old = len(bi_word)
    bi_word.append(row[0].split(" "))
    new = len(bi_word)


final_tags = []
first_tag = []
second_tag = []
for ind in range(len(bi_word)):
  final_tags.append(bigram_tagger.tag(bi_word[ind]))


for i in range(len(final_tags)):
  for j in range(len(final_tags[i])):
    if j%2==0:
      first_tag.append(final_tags[i][j][1])
    else:
      second_tag.append(final_tags[i][j][1])


k = len(first_tag)
bigram_note = []
for i in range(k):
  if first_tag[i] == 'n' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'n' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adv' and second_tag[i]=='v':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])
  elif first_tag[i] == 'adj' and second_tag[i]=='n':
    bigram_note.append(final_tags[i][0][0]+" "+final_tags[i][1][0])

print("Total number of bi-words in the file: " + str(len(final_tags)))
print("Calculated Meaningful bigrams: " + str(len(bigram_note)))
print("The Meaningful Bigrams: ")
print(bigram_note)

#for i in bigram_note:
#  print(bi_word[i]+" "+bi_word[i+1])

Total number of bi-words in the file: 12000
Calculated Meaningful bigrams: 2953
The Meaningful Bigrams: 
['कंप्यूटर गेम', 'नेहरू परिवार', 'दुर्गा मन्दिर', 'राजस्थान राज्य', 'व्यास नदी', 'तीर्थ स्थल', 'स्वर्ण पत्र', 'नागरिक सेना', 'शिशु विशेषज्ञ', 'जीव जंतु', 'संगीत आचार्य', 'प्रशांत महासागर', 'अध्ययन केन्द्र', 'नीलगिरि श्रृंखला', 'रिक्शा चालक', 'ध्यान केन्द्र', 'नेहरू बाजार', 'वन संसार', 'हाथी दांत', 'अजमेर पर्यटन', 'राम मंदिर', 'वर्तमान युग', 'हॉकी खिलाड़ी', 'क्रिकेट मैच', 'नेहरू पार्क', 'आपात बैठक', 'कला केंद्र', 'सर्राफा बाजार', 'आज्ञा पत्र', 'जावा द्वीप', 'केन्द्र सरकार', 'भगवान दत्तात्रेय', 'चन्द्र भगवान', 'टेस्ट मैच', 'खाड़ी युद्ध', 'कृषि परिषद', 'चीनी उद्योग', 'कलिंग साम्राज्य', 'जमीन विवाद', 'यात्रा उद्योग', 'ब्राह्मणी चील', 'भारत सरकार', 'सब्जी बाजार', 'नृत्य संगीत', 'बुनाई केन्द्र', 'कृषि केन्द्र', 'शोध समिति', 'सहायता सामग्री', 'पाचन तंत्र', 'चेतावनी तंत्र', 'जल जीवशाला', 'तांडव नृत्य', 'सूर्य भगवान', 'सिंधु घाटी', 'संचार प्रणाली', 'चीन सागर', 'शब्द कोष', 'तीस्ता नदी', 'नेहर

In [ ]:
#Pos_tagger for given text/paragraph

tag = []
bi_word = []
dataset = []
tags = []
words = []
with open("/content/drive/MyDrive/sharesd/sms-cw_2/testoneHindi.txt") as file:
  dataset = file.readlines()

words.append(dataset[0].split(" "))

tag.append(pos_tagger.tag(words[0]))

for i in tag:
  for j in i:
    tags.append(j[1])

for i in tag:
  for j in i:
    bi_word.append(j[0])

k = len(tags)-1
bigram_note = []
for i in range(k):
  if tags[i] == 'n' and tags[i+1]=='n':
    bigram_note.append(i)
  elif tags[i] == 'n' and tags[i+1]=='v':
    bigram_note.append(i)
  elif tags[i] == 'adj' and tags[i+1]=='v':
    bigram_note.append(i)
  elif tags[i] == 'adv' and tags[i+1]=='n':
    bigram_note.append(i)
  elif tags[i] == 'adv' and tags[i+1]=='v':
    bigram_note.append(i)
  elif tags[i] == 'adj' and tags[i+1]=='n':
    bigram_note.append(i)

final_tags = []
for i in bigram_note:
  final_tags.append(bi_word[i]+" "+bi_word[i+1])


print("Total number of words in the text file: " + str(k))
print("Calculated Meaningful bigrams: " + str(len(bigram_note)))
print("The Meaningful Bigrams: ")

print(final_tags)

#for i in bigram_note:
#  print(bi_word[i]+" "+bi_word[i+1])

Total number of words in the text file: 376
Calculated Meaningful bigrams: 38
The Meaningful Bigrams: 
['विश्व स्वास्थ्य', 'स्वास्थ्य संगठन', 'घोषित कर', 'कर दिया', 'संक्रमण बहुत', 'एक व्यक्ति', 'व्यक्ति में', 'बचाव के', 'सरकार द्वारा', 'की सलाह', 'व्यक्ति में', 'में बहुत', 'की सलाह', 'पर जोर', 'देश में', 'के लक्षण', 'की बात', 'नाक बहना', 'वायरस एक', 'व्यक्ति में', 'में बहुत', 'चीन में', 'बचाव के', 'बचाव के', 'पूरा ध्यान', 'ध्यान रखना', 'की जगह', 'जगह नमस्ते', 'की आदत', 'घर में', 'में लाई', 'नहीं करना', 'की आदत', 'कम 1', '1 बार', 'बार हल्दी', 'की आदत', 'की जांच']


In [14]:
#Pos_tagger for given text/paragraph

def Meaningful_Bigrams(dataset):
  tag = []
  bi_word = []
  tags = []
  words = []
  words.append(dataset[0].split(" "))

  tag.append(pos_tagger.tag(words[0]))

  for i in tag:
    for j in i:
      tags.append(j[1])

  for i in tag:
    for j in i:
      bi_word.append(j[0])

  k = len(tags)-1
  bigram_note = []
  for i in range(k):
    if tags[i] == 'n' and tags[i+1]=='n':
      bigram_note.append(i)
    elif tags[i] == 'n' and tags[i+1]=='v':
      bigram_note.append(i)
    elif tags[i] == 'adj' and tags[i+1]=='v':
      bigram_note.append(i)
    elif tags[i] == 'adv' and tags[i+1]=='n':
      bigram_note.append(i)
    elif tags[i] == 'adv' and tags[i+1]=='v':
      bigram_note.append(i)
    elif tags[i] == 'adj' and tags[i+1]=='n':
      bigram_note.append(i)

  final_tags = []
  for i in bigram_note:
    final_tags.append(bi_word[i]+" "+bi_word[i+1])
  return(final_tags)
'''
  print("Total number of words in the text file: " + str(k))
  print("Calculated Meaningful bigrams: " + str(len(bigram_note)))
  print("The Meaningful Bigrams: ")

  print(final_tags)
  return(final_tags)
'''
  
#for i in bigram_note:
#  print(bi_word[i]+" "+bi_word[i+1])

'\n  print("Total number of words in the text file: " + str(k))\n  print("Calculated Meaningful bigrams: " + str(len(bigram_note)))\n  print("The Meaningful Bigrams: ")\n\n  print(final_tags)\n  return(final_tags)\n'

In [ ]:
dataset = []

with open("/content/drive/MyDrive/sharesd/sms-cw_2/testoneHindi.txt") as file:
  dataset = file.readlines()

Meaningful_Bigrams(dataset)

['विश्व स्वास्थ्य',
 'स्वास्थ्य संगठन',
 'घोषित कर',
 'कर दिया',
 'संक्रमण बहुत',
 'एक व्यक्ति',
 'व्यक्ति में',
 'बचाव के',
 'सरकार द्वारा',
 'की सलाह',
 'व्यक्ति में',
 'में बहुत',
 'की सलाह',
 'पर जोर',
 'देश में',
 'के लक्षण',
 'की बात',
 'नाक बहना',
 'वायरस एक',
 'व्यक्ति में',
 'में बहुत',
 'चीन में',
 'बचाव के',
 'बचाव के',
 'पूरा ध्यान',
 'ध्यान रखना',
 'की जगह',
 'जगह नमस्ते',
 'की आदत',
 'घर में',
 'में लाई',
 'नहीं करना',
 'की आदत',
 'कम 1',
 '1 बार',
 'बार हल्दी',
 'की आदत',
 'की जांच']

In [ ]:
#bigram tagger for given text/paragraph
tag = []
bi_word = []
dataset = []
tags = []
words = []
with open("/content/drive/MyDrive/sharesd/sms-cw_2/testoneHindi.txt") as file:
  dataset = file.readlines()

words.append(dataset[0].split(" "))

tag.append(bigram_tagger.tag(words[0]))

for i in tag:
  for j in i:
    tags.append(j[1])

for i in tag:
  for j in i:
    bi_word.append(j[0])

k = len(tags)-1
bigram_note = []
for i in range(k):
  if tags[i] == 'n' and tags[i+1]=='n':
    bigram_note.append(i)
  elif tags[i] == 'n' and tags[i+1]=='v':
    bigram_note.append(i)
  elif tags[i] == 'adj' and tags[i+1]=='v':
    bigram_note.append(i)
  elif tags[i] == 'adv' and tags[i+1]=='n':
    bigram_note.append(i)
  elif tags[i] == 'adv' and tags[i+1]=='v':
    bigram_note.append(i)
  elif tags[i] == 'adj' and tags[i+1]=='n':
    bigram_note.append(i)

final_tags = []
for i in bigram_note:
  final_tags.append(bi_word[i]+" "+bi_word[i+1])


print("Total number of words in the text file: " + str(k))
print("Calculated Meaningful bigrams: " + str(len(bigram_note)))
print("The Meaningful Bigrams: ")

print(final_tags)

#for i in bigram_note:
#  print(bi_word[i]+" "+bi_word[i+1])


Total number of words in the text file: 376
Calculated Meaningful bigrams: 0
The Meaningful Bigrams: 
[]


In [15]:
!pip install flask-ngrok

In [21]:
from flask import Flask, request, render_template
#import spacy
from flask_ngrok import run_with_ngrok

app = Flask(__name__, template_folder= '/content/drive/MyDrive/sharesd/sms-cw_2')
run_with_ngrok(app)

@app.route('/', methods=['GET', 'POST'])
def homepage():
  #return "<h1>GFG is great platform to learn</h1>"
  return render_template('index.html')

@app.route('/pos_tag', methods=['GET', 'POST'])
def pos_tags():
  doc = Meaningful_Bigrams(request.form['text'].strip())
  response = doc
  doc = sim_words_in_dataset_inputtext(request.form['text'].strip(),bi_wordss)
  #tokens = [token.text for token in doc]
  #pos = [token.pos_ for token in doc]      
  #data = listOfTuples(tokens, pos)   
  response.append(doc)
  return render_template('index.html', data=request.form['text'], response=response)

if __name__ == '__main__':
  app.run()
#हिंदी में टायपिंग करना 20वां वर्ष बहुत आसान बना
# 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://686f44b22c64.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Apr/2021 06:19:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 06:19:02] "GET /favicon.ico HTTP/1.1" 404 -


127.0.0.1 - - [14/Apr/2021 06:20:58] "POST /pos_tag HTTP/1.1" 200 -


In [ ]:
from google.colab import drive
drive.mount('/content/drive')